# Book Recommendation Hackathon 

**Task:** Rank 20 editions for each user from 200 candidates, optimizing Score = 0.7×NDCG@20 + 0.3×Diversity@20

**Strategy - classic, catboost ranker + rearranging (for the 30% of the residual metric bcs catboost is fitted on ndcg)**

In [53]:
import sys 
import os
import warnings 

os.environ['OPENBLUS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [54]:
import pandas as pd
import datetime as dt
import numpy as np

interactions = pd.read_csv('data/interactions.csv')
editions = pd.read_csv('data/editions.csv')
users = pd.read_csv('data/users.csv')
book_genres = pd.read_csv('data/book_genres.csv')
genres = pd.read_csv('data/genres.csv')
authors = pd.read_csv('data/authors.csv') 
target_users = pd.read_csv('submit/targets.csv') 
target_interactions = pd.read_csv('submit/candidates.csv')

print('all data frames have been loaded successfully')

all data frames have been loaded successfully


In [55]:
%%time

interactions['event_ts'] = pd.to_datetime(interactions['event_ts'])

split_date = pd.Timestamp('2025-03-12')

feature_source = interactions.loc[interactions['event_ts'] < split_date]
train = interactions.loc[interactions['event_ts'] > split_date]

CPU times: user 35.2 ms, sys: 13.1 ms, total: 48.3 ms
Wall time: 62.7 ms


In [56]:
%%time

book_genres = book_genres.groupby('book_id')['genre_id'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
enriched_editions = editions.merge(book_genres, on='book_id')

enriched_editions['author_productivity']= enriched_editions.author_id.map(enriched_editions.author_id.value_counts())

feature_source = feature_source.drop('event_ts', axis=1)
feature_source = feature_source.merge(users, on='user_id')
feature_source = feature_source.merge(enriched_editions, on='edition_id')

feature_source = feature_source.drop('book_id', axis=1) #1 to 1 with edition_id
feature_source = feature_source.drop('publisher_id', axis=1) #1 to 1 with edition_id

feature_source['edition_popularity_score'] = feature_source.edition_id.map(feature_source.edition_id.value_counts())
feature_source['reader_mean_age'] = feature_source.groupby('edition_id')['age'].transform('mean')
feature_source['book_age'] = 2026 - feature_source['publication_year']
feature_source['user_mean_rating'] = feature_source.groupby('user_id')['rating'].transform('mean')
feature_source['book_mean_rating'] = feature_source.groupby('edition_id')['rating'].transform('mean')
feature_source = feature_source.drop('rating', axis=1)

user_cols = ['user_id', 'gender', 'age', 'user_mean_rating']

user_features = feature_source[user_cols].drop_duplicates().reset_index()
user_features = user_features.drop('index', axis=1)

book_features = feature_source[[f for f in feature_source.columns.to_list() if f not in user_cols]].drop_duplicates().reset_index()
book_features = book_features.drop(['event_type', 'index'], axis=1)
book_features = book_features.drop_duplicates()

train = train.merge(book_features, on='edition_id').merge(user_features, on='user_id')
train = train.drop(['event_ts', 'rating'], axis=1)

CPU times: user 2.44 s, sys: 450 ms, total: 2.89 s
Wall time: 2.94 s


In [57]:
train.head(1)

,user_id,edition_id,event_type,author_id,publication_year,age_restriction,language_id,title,description,genre_id,author_productivity,edition_popularity_score,reader_mean_age,book_age,book_mean_rating,gender,age,user_mean_rating
0,560,1010822636,2,507926.0,2024,16,119,Призраки белых ночей,На встрече бывших однокурсников Александра ста...,1222 1224 1309,56,7,37.857143,2,7.571429,2.0,9.0,7.733333


In [58]:
%%time

NEGATIVE_FRACTION = 3

needed_cols = [
    'user_id',
    'edition_id',
    'event_type',
    'gender',
    'age',
    'author_id',
    'publication_year',
    'age_restriction',
    'language_id',
    'title',
    'description',
    'genre_id',
    'author_productivity',
    'edition_popularity_score',
    'reader_mean_age',
    'book_age',
    'user_mean_rating',
    'book_mean_rating'
 ]

num_negative_samples_per_user = train['user_id'].value_counts() * NEGATIVE_FRACTION

max_books = len(book_features)
num_negative_samples_per_user = num_negative_samples_per_user.clip(upper=max_books)

user_cols = ['user_id', 'gender', 'age', 'user_mean_rating']
user_features = train[user_cols].drop_duplicates(subset=['user_id']).set_index('user_id')

user_ids = num_negative_samples_per_user.index.to_numpy()
counts = num_negative_samples_per_user.to_numpy()

repeated_user_ids = np.repeat(user_ids, counts)

temp_df = pd.DataFrame({'user_id': repeated_user_ids})
book_indices = temp_df.groupby('user_id', sort=False).cumcount().to_numpy()

book_features = book_features.sort_values('edition_popularity_score', ascending=False).reset_index(drop=True)

sampled_books = book_features.iloc[book_indices].reset_index(drop=True)
sampled_users = user_features.loc[repeated_user_ids].reset_index(drop=True)

negativity_builder = pd.concat([sampled_users, sampled_books], axis=1)

negativity_builder['user_id'] = repeated_user_ids # Ensures the ID column is retained
negativity_builder['event_type'] = 0

negativity_builder = negativity_builder[needed_cols]

train = pd.concat([train, negativity_builder], ignore_index=True)

train = train.sort_values(
    by='event_type', 
    key=lambda x: x != 0
)

train = train.drop_duplicates(subset=['user_id', 'edition_id'], keep='first')
train.edition_id = train.edition_id.astype(str)
train = train.reset_index().drop('index', axis=1)

CPU times: user 54.6 ms, sys: 24.1 ms, total: 78.7 ms
Wall time: 95 ms


In [59]:
train.head(3)

,user_id,edition_id,event_type,author_id,publication_year,age_restriction,language_id,title,description,genre_id,author_productivity,edition_popularity_score,reader_mean_age,book_age,book_mean_rating,gender,age,user_mean_rating
0,4858510,1010816570,0,1202004.0,2024,16,119,Хороших девочек не убивают,Пять лет назад популярную школьную красавицу Э...,125 127 149,19,125,29.241935,2,8.602564,2.0,34.0,8.266667
1,1808500,1008788845,0,2386468.0,2023,12,119,Граф Аверин. Колдун Российской империи,"Магический Петербург, 1982 год.\r\nГраф Аверин...",1240 1244 1246 1251,33,248,33.016393,3,9.187500,NaN,36.0,9.140351
2,1808500,1010122669,0,2386468.0,2024,16,119,Тайна мертвого ректора. Книга 1,После грандиозной и кровопролитной битвы граф ...,1243 1244,33,304,33.577181,2,9.018349,NaN,36.0,9.140351


In [88]:
# 1. Сначала готовим ЕДИНЫЙ датасет с правильным порядком
# Сбрасываем индекс СРАЗУ, чтобы он был 0, 1, 2... везде
train = train.sort_values('user_id').reset_index(drop=True)

# 2. Выделяем группы (Queries) и таргет (Label) ПЕРЕД удалением колонок
queries = train['user_id']
label = train['event_type']

# 3. Готовим признаки (X) - удаляем ВСЁ лишнее тут
# ВАЖНО: Убираем user_id отсюда, чтобы не было лика!
drop_cols = ['event_type', 'edition_id', 'user_id', 
             'edition_popularity_score', 'user_mean_rating', 
             'book_mean_rating', 'reader_mean_age']          
X = train.drop(columns=drop_cols)

# 4. Приводим категории к строкам
cat_features_safe = ['author_id', 'language_id', 'gender', 'genre_id'] 
for col in cat_features_safe:
    X[col] = X[col].astype(str).replace('nan', 'unknown')

# 5. МАГИЧЕСКИЙ СПЛИТ: Передаем X, y и queries ВМЕСТЕ
# shuffle=False обязателен для ранкера, чтобы не разорвать группы юзеров
X_train, X_test, y_train, y_test, q_train, q_test = train_test_split(
    X, label, queries, 
    test_size=0.33, 
    random_state=42, 
    shuffle=False 
)

# 6. Проверка на вшивость (Обязательно посмотри этот принт!)
print(f"Размер групп в трейне: {q_train.value_counts().mean():.2f} книг на юзера")
# Если тут будет 1.0 — значит данные кривые. Должно быть 5-10-20.

# Заполни пустоты во ВСЕХ текстовых признаках
text_cols = ['title', 'description', 'genre_id'] # проверь список по своему коду

for col in text_cols:
    X_train[col] = X_train[col].fillna('none').astype(str)
    X_test[col] = X_test[col].fillna('none').astype(str)

# Теперь создавай Pool
train_pool = Pool(
    data=X_train,
    label=label_train,
    group_id=q_train,
    cat_features=cat_features,
    text_features=text_features
)

val_pool = Pool(
    data=X_test, label=y_test, group_id=q_test,
    cat_features=cat_features_safe, text_features=text_cols
)

Размер групп в трейне: 26.57 книг на юзера


In [89]:
%%time

from catboost import CatBoostRanker

TASK_TYPE = 'CPU'

model = CatBoostRanker(
    iterations=1000, 
    learning_rate=0.1, 
    loss_function='YetiRank', 
    eval_metric='NDCG:top=20', 
    random_seed=42, 
    task_type=TASK_TYPE, 
    metric_period=100, 
    use_best_model=True, 
    early_stopping_rounds=100
)

model.fit(
    train_pool,
    eval_set=val_pool, 
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Groupwise loss function. OneHotMaxSize set to 10


0:	test: 0.2957117	best: 0.2957117 (0)	total: 230ms	remaining: 3m 49s
100:	test: 0.2719063	best: 0.2957117 (0)	total: 19.1s	remaining: 2m 49s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2957117316
bestIteration = 0

Shrink model to first 1 iterations.
CPU times: user 2min 4s, sys: 5.19 s, total: 2min 9s
Wall time: 21 s


In [79]:
# 1. Проверяем, что модель обучена. Если выведет True, значит всё ок.
print(f"Is model trained: {model.is_fitted()}")

# 2. Явно запрашиваем важность
importances = model.get_feature_importance(train_pool)
feature_names = model.feature_names_

# 3. Собираем таблицу
fea_imp = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(fea_imp)

Is model trained: True
                feature  importance
5           description    0.016096
4                 title    0.007363
10                  age    0.002044
7   author_productivity    0.001471
0             author_id    0.001300
3           language_id    0.000212
2       age_restriction    0.000159
9                gender   -0.000020
1      publication_year   -0.000204
8              book_age   -0.000240
6              genre_id   -0.009451
